<a href="https://colab.research.google.com/github/aljeshishe/notebooks/blob/master/template_vgg_gpu_v2_with_wb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt 
import sys 

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from time import time 

In [0]:
import os
# use gpu/cpu/tpu
# see details in https://colab.research.google.com/drive/1cpuwjKTJbMjlvZ7opyrWzMXF_NYnjkiE#scrollTo=y3gk7nSvTUFZ
gpus = tf.config.experimental.list_physical_devices('GPU')
COLAB_TPU_ADDR = os.environ.get('COLAB_TPU_ADDR')
if COLAB_TPU_ADDR:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + COLAB_TPU_ADDR)
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
  print('Running on TPU ')  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

def notebook_name():
  import re
  import ipykernel
  import requests

  from notebook.notebookapp import list_running_servers
  # kernel_id = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
  for ss in list_running_servers():
      response = requests.get(f'{ss["url"]}api/sessions',params={'token': ss.get('token', '')})
      return response.json()[0]['name']

import pathlib

import os
project, _, _ = notebook_name().rpartition('.')
working_dir = f'/content/drive/My Drive/Colab Notebooks/{project}'
print(f'Current project: {project}')
print(f'Places at: {working_dir}')
pathlib.Path(working_dir).mkdir(parents=True, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current project: template_vgg_gpu_v2_with_wb
Places at: /content/drive/My Drive/Colab Notebooks/template_vgg_gpu_v2_with_wb


In [0]:
pip install wandb -q

In [0]:
#load dataset
(trainX, trainY),(testX, testY) = cifar10.load_data()

#one hot encode the target 
trainY = keras.utils.to_categorical(trainY)
testY = keras.utils.to_categorical(testY)

# normalize the data
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

v_split = 0.5
index_of_validation = int(v_split * len(testX))
validX = np.asarray(testX[-index_of_validation:])
validY = np.asarray(testY[-index_of_validation:])
testX = np.asarray(testX[:-index_of_validation])
testY = np.asarray(testY[:-index_of_validation])
print(f'Prepared dataset:')
print(f'trainX:{trainX.shape}')
print(f'trainY:{trainY.shape}')
print(f'validX:{validX.shape}')
print(f'validY:{validY.shape}')
print(f'testX:{testX.shape}')
print(f'testY:{testY.shape}')


Prepared dataset:
trainX:(50000, 32, 32, 3)
trainY:(50000, 10)
validX:(5000, 32, 32, 3)
validY:(5000, 10)
testX:(5000, 32, 32, 3)
testY:(5000, 10)


In [0]:
import wandb
from wandb.keras import WandbCallback
config = dict(
  batch_size = 64,          # input batch size for training (default: 64)
  epochs = 1000,             # number of epochs to train (default: 10)
  lr = 0.001,               # learning rate (default: 0.01)
  momentum = 0.9,          # SGD momentum (default: 0.5) 
  seed = 42,               # random seed (default: 42)
  log_interval = 10,     # how many batches to wait before logging training status
  weight_decay = 0.0005
)
wandb.init(project=project, dir=working_dir, config=config)
config = wandb.config

In [0]:
def vgg():
  input_shape = (trainX.shape[1], trainX.shape[2], 3)
  
  # Define the model architecture - This is a simplified version of the VGG19 architecture
  model = Sequential()
  
  # Set of Conv2D, Conv2D, MaxPooling2D layers with 32 and 64 filters
  model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))

  model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 128 filters
  model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 256 filters
  model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 512 filters
  model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 512 filters
  model.add(Conv2D(filters = 1024, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 1024, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Flatten
  model.add(Flatten())
  model.add(Dense(512, activation ='relu', kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  num_classes = 10
  model.add(Dense(num_classes, activation = "softmax"))
  opt = SGD(lr = config.lr, momentum=config.momentum)
  model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [0]:
# define model
def baseline():
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same', input_shape = (32,32,3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.4))
   

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = 'softmax'))
    
    #compile model 
    opt = SGD(lr = config.lr, momentum=config.momentum)
    model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model


In [0]:
with strategy.scope():
  model = baseline()

#create data generator 
datagen = ImageDataGenerator(width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True)

#iterator 
train = datagen.flow(trainX, trainY, batch_size=config.batch_size)

# fit model
steps = int(trainX.shape[0]/ 64)
history = model.fit_generator(train,
                              steps_per_epoch=steps,
                              epochs=config.epochs, validation_data=(validX, validY), verbose=1,
                              callbacks=[WandbCallback(validation_data=(validX, validY),
                                                       save_model=True, verbose=1)])



In [0]:
# restore a model file from a specific run by user "vanpelt" in "my-project"
best_model = wandb.restore('model-best.h5')
model.load_weights(best_model.name)

In [0]:
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))


In [0]:
wandb.save("mymodel.h5")
model.save(os.path.join(wandb.run.dir, "mymodel.h5"))